In [ ]:
!nvidia-smi

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
import logging
import numpy as np
import torch, os
from sicgan.utils.metrics import compare_meshes
from tqdm import tqdm

from IPython.core.display import display, HTML
from sicgan.config import Config
from sicgan.models import Pixel2MeshHead
from sicgan.models import GraphConvClf
from mpl_toolkits.mplot3d import Axes3D
from sicgan.models import MeshLoss
from pytorch3d.io import load_obj, save_obj
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.animation #import FuncAnimation
from matplotlib.animation import FuncAnimation


from sicgan.data.build_data_loader import build_data_loader
display(HTML("<style>.container { width:80% !important; }</style>"))
device = torch.device("cuda:0")

In [4]:
from PyGEL3D import gel
from PyGEL3D import js
def plot_mesh(mesh):
    save_obj('mesh.obj', mesh.verts_packed(), mesh.faces_packed())
    js.set_export_mode()
    m = gel.obj_load('mesh.obj')
    js.display(m, smooth=False)

In [20]:
_C = Config('config/train_p2m.yml',[])

In [21]:
data_loader = build_data_loader(_C, "MeshVox", split_name='test', multigpu=False)
len(data_loader)

1500

In [7]:
cp = torch.load('pixel2mesh_R50.pth')["best_states"]["model"]
from collections import OrderedDict
new_cp = OrderedDict()
for k, v in cp.items():
    name = k[7:] # remove `module.`
    if name != 'K':
        new_cp[name] = v

In [9]:
model = Pixel2MeshHead(_C).cuda()
model.load_state_dict(new_cp)

<All keys matched successfully>

In [22]:
for batch in tqdm(data_loader):
    batch = data_loader.postprocess(batch, device)
    imgs, meshes_gt = batch[0], batch[1]
    break

  0%|          | 0/1500 [00:01<?, ?it/s]


In [23]:
imgs.shape

torch.Size([32, 3, 137, 137])

In [24]:
_,m = model(imgs)

In [25]:
m

In [ ]:
plot_mesh(meshes_gt)

In [ ]:
plot_mesh(meshes_pred[-1])

# Evaluation  - Change this

In [ ]:
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.loss import (
    chamfer_distance, 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
)

In [ ]:
device = torch.device("cuda:0")
class_names = {
    "03001627": "chair",
    "04379243": "table",
}

num_instances = {i: 0 for i in class_names}
chamfer = {i: 0 for i in class_names}
# normal = {i: 0 for i in class_names}
f1_1e_4 = {i: 0 for i in class_names}
f1_2e_4 = {i: 0 for i in class_names}

In [ ]:
loss_fn_kwargs = {
    "chamfer_weight": 1,
    "normal_weight": 0,
    "edge_weight": 0,
    "gt_num_samples": 5000,
    "pred_num_samples": 5000,
}

mesh_loss = MeshLoss(**loss_fn_kwargs).cuda()

In [ ]:
sample_trg = sample_points_from_meshes(meshes_gt, 5000)
sample_src = sample_points_from_meshes(meshes_pred[-1], 5000)
chamfer_distance(sample_trg, sample_src)

In [ ]:
num_batch_evaluated = 0
cd_chair = []
cd_table = []
for batch in tqdm(data_loader):
    batch = data_loader.postprocess(batch, device)
    imgs, meshes_gt, _, _, id_strs = batch
#     sids = [id_str.split("-")[0] for id_str in id_strs]
    sid = id_strs[0].split("-")[0]
#     for sid in sids:
#         num_instances[sid] += 1

    # with inference_context(model):
    
    # Change this
    _,meshes_pred = model(imgs)     #Removed Voxels
    
    loss, _ = mesh_loss(meshes_gt, meshes_pred[-1])
    if sid == '03001627':
        cd_chair.append(loss.item())
    elif sid == '04379243':
        cd_table.append(loss.item())


#     cur_metrics = compare_meshes(
#         meshes_pred[-1], meshes_gt, scale=0.57, thresholds=[0.01, 0.014142], reduce=False
#     )
#     cur_metrics["verts_per_mesh"] = meshes_pred[-1].num_verts_per_mesh().cpu()
#     cur_metrics["faces_per_mesh"] = meshes_pred[-1].num_faces_per_mesh().cpu()

#     for i, sid in enumerate(sids):
#         chamfer[sid] += cur_metrics["Chamfer-L2"][i].item()
#         # normal[sid] += cur_metrics["AbsNormalConsistency"][i].item()
#         f1_1e_4[sid] += cur_metrics["F1@%f" % 0.01][i].item()
#         f1_2e_4[sid] += cur_metrics["F1@%f" % 0.014142][i].item()

#     num_batch_evaluated += 1
#     print("Evaluated %d / %d batches" % (num_batch_evaluated, len(data_loader)))

In [ ]:
{"chamfer": chamfer, "f1_1e_4": f1_1e_4, "f1_2e_4": f1_2e_4}

In [ ]:
import numpy as np

In [ ]:
np.mean(cd_chair), np.mean(cd_table)